<a href="https://colab.research.google.com/github/deangarcia/NLP/blob/main/NLP_Assignment1_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment, you will familiarize yourself with:



*   Python
*   Jupyter Notebooks
*   Google Colab

to develop

* A n-gram based language model, with smoothing

to be able to

* Produce naturalish text
* Rate the perplexity of a text given your model

First we will load in some data.

Provided is code that will download a file and rename it corpus.zip

In [ ]:
!wget -O corpus.zip https://digitalrepository.wheatoncollege.edu/bitstream/handle/11040/24451/Shakespeare%20Corpus%20by%20Play.zip?sequence=1&isAllowed=y 

By typing "!" in a notebook, you can use command line prompts

In [ ]:
!ls 


Now, we will unzip the corpus


In [ ]:
!unzip -o corpus.zip

We can even have python and the command line interact.

In [4]:
files = !ls Shake*/*/*_noSCRB.txt 


`files` is now a list containing all of the files we wish to use in our data

In [ ]:
print(files)

Note, there are extra quotes on the files, so we will use list comprehension to remove those

In [6]:
files = [file[1:-1] for file in files]

#STEP 1


* Go through all of the files
  * Load the file `file = open(filename,'r')` 
  * Use the spacy tokenizer to tokenize the text `tokens = tokenizer(file.read())`
  * Loop through the tokens and make everything lowercase `.lower()`
  * Use all of this to make a list (for each file) of lists of strings (the tokens in each file) called `texts` 



  e.g.

  If we read in one file that looked like:
  
    HAMLET: Alas poor Yorick.

    GRAVEDIGGER: Put that skull down.



  The end results should look like:

  `texts = [['<s>', 'HAMLET:', 'Alas', 'poor', 'Yorick.','\n','GRAVEDIGGER:', Put','that', 'skull', 'down.', '</s>']]`
 

In [ ]:
!pip install spacy

In [ ]:
import re
import spacy
from spacy.lang.en import English
nlp = English()
from spacy.tokenizer import Tokenizer

special_cases = {":)": [{"ORTH": ":)"}]}
prefix_re = re.compile(r'''^[\[\("']''')
suffix_re = re.compile(r'''[\]\)"']$''')
infix_re = re.compile(r'''[-~]''')

def custom_tokenizer(nlp):
    return Tokenizer(nlp.vocab, rules=special_cases,
                                prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer)
    
nlp = spacy.load("en_core_web_sm")
nlp.tokenizer = custom_tokenizer(nlp)
tokenizer = nlp.tokenizer
texts = []

for filename in files:
  test = []
  filex = open(filename,'r')
  tokens = tokenizer(filex.read())
  for token in tokens:
    test.append(token.lemma_.lower())
  texts.append(test)




#STEP 2

With our files read in, it's time to do some counting!

* Implement the function `get_counts(texts,n)` which takes in a list of list of strings -- `texts` -- and the arity of the n-gram -- `n` 
* `get_counts` should return a dictionary where the keys are the n-grams (as tuples of strings) and the values are the integer counts of all of the found n-grams


Here's a quick primer on tuples in Python

    triple = (1,2,3)
    double = (1,2)
    single = (1,) #note the trailing comma to distinguish from parenethesis 
    null_tuple = tuple() 

    list_triple = [1,2,3]
    tuple_triple = tuple(list_triple)
    


In [9]:
def get_counts(texts,n):
  ngrams = {}
  for text in texts:
    for i in range(len(text) - n + 1):
      temp = [] 
      for j in range(n):
        temp.append(text[i+j])
      tuple_temp = tuple(temp)
      if tuple_temp in ngrams.keys():
        ngrams[tuple_temp] += 1
      else:
        ngrams[tuple_temp] = 1

  return ngrams

unigrams = get_counts(texts,1)
bigrams = get_counts(texts,2)
trigrams = get_counts(texts,3)


#STEP 3

With our counts, it's time to make our first language model.

* Implement the class NGramLM 
  * Initialize the LM
    * First use `get_counts` to get the raw counts of the n-grams up to order `n`
    * Convert these raw counts into probabilities
      * ### Question 1:  What is the relationship between raw counts of n-grams of order n and n-1 and the probabilities of the next word for order n? 
  * Implement the `generate_text` function
    * You are given an optional prompt (a list of strings) -- first convert these to lower case (as the original text was)
    * You should return a list of strings based on randomly choosing according to the probabilities of the language model (using the `random` library)
  * Implement `score_text` which when given a text (a list of strings) returns the perplexity of that text
    * Make sure you don't have any divide by 0 (or `log(0)`) issues -- return `float('inf')` if there is an n-gram not found in the model (note: the perplexity is infinite, not the probability)

In [151]:
import random
random.seed(0)

class NGramLM():
  def __init__(self,texts,n):
    self.xgram_probs = {}
    if n == 1:
      xgram = get_counts(texts,n)
      for key in xgram:
        self.xgram_probs[key] = xgram[key] / len(xgram)
    else:
      ngram = get_counts(texts,n)
      nmI_gram = get_counts(texts,n-1)
      for key in ngram:
        self.xgram_probs[key] = ngram[key] / nmI_gram[key[:n-1]]
          
  def generate_text(self, length, prompt=[]):
    temp = []

    for i in range(len(prompt)):
      temp.append(prompt[i].lower())
    
    for i in range(0, length):
      r = random.random()
      # subset of the dictionary that contains only keys that contain the last prompt[n-1] to prompt[last index]
      subset = {}
      if len(prompt) > 1:
        temp_tup = ""
        for tup in self.xgram_probs.keys():
          if len(tup) > 1:
            # should extract the last n values of list and make it a tup and check if that subset of a tup exists in the tup.
            if temp[len(temp) - len(tup)] in tup:
              subset[tup] = self.xgram_probs[tup]
          else:
            subset[tup] = self.xgram_probs[tup]

        close_match = 1
        for tup in sorted(subset):
          if abs(r - self.xgram_probs[tup]) < close_match:
            close_match = abs(r - self.xgram_probs[tup])
            temp_tup = tup
              
        temp += list(temp_tup)
          
      else:
        temp_tup = ""
        close_match = 1
        for tup in self.xgram_probs.keys():
          if abs(r - self.xgram_probs[tup]) < close_match:
            close_match = abs(r - self.xgram_probs[tup])
            temp_tup = tup
            
        temp += list(temp_tup)

    return temp

  def score_text(self,text):
    raise NotImplementedError("STEP 3: Given a text, return the perplexity of that text ")


In [152]:
for n in [1,2]:
  print(f'Order {n} LM:')
  lm = NGramLM(texts,n)
  
  print(' '.join(lm.generate_text(100)))
  print("next")
  print(' '.join(lm.generate_text(100,['palm', 'to'])))

  #print(lm.score_text(['IBM','announced','today','that','they','will','be','buying','Google']))
  #print(lm.score_text(['palm', 'to', 'palm', 'is', 'holy', 'palmers']))
  #print(lm.score_text(['what','do','you','say','to','that','Hamlet']))

Order 1 LM:

   
   a you and i 
   of be 
   
   and you 
   
   you 
   
     
   
   of 
   
   
   be thou a 
   
   
     be 
   you 
   the stand 
   i 
   
   widow be 
   my of 
   
 
   my 
     
   a 

 of and 
   thou the 
   the 
   the 
     
   
   a 
   to 
   you 
 
 
   
   be will 
   
   
   
   
   
   the to 
   you 
   
   
   
   
     
   be 
   
     
   
   

 
   be
next
palm to 
   
   my 
   it that 
   of 
   thou ' 
   come 
   
   and 
   them 
   
   
   to of 
     give who 
     
 he in 
   
     us a thou you that 
   of 
 and when thou 
     in of 
   
   
   
 
   
     no 
   
   of you 
   a 
 be i 
   and of of 
   you the till 
   of would you my 
     of you of 
     
   
   
   to a 
   

          in of my 
   to 
   
   a i 
   a 
   or
Order 2 LM:
so. 
   remedy. 
     sin 
     full of eyes; 
     image of 

                      enter pair of v. scene me; 
     out. 
   




 scene aweary of fool? 
   ho! 
   1603 

 afar off death be 



#STEP 4

Build a better language model

* Implement the function `truncate_texts` which takes in texts and a threshold
  * Replace all words with counts less than the threshold with `'<UNK>'`
* Implement the function `get_vocabulary` which takes in texts and returns a set of all of the words found in the texts
* Implement the UnknownLM language model
  * Handle unknown vocabulary gracefully by checking the vocabulary and replacing with `'<UNK>'`



In [ ]:
def truncate_texts(texts,threshold):
  modified_texts = []
  raise NotImplementedError("STEP 4: Modify the texts by replacing all words with counts less than the threshold with '<UNK>'")
  return modified_texts

def get_vocabulary(texts):
  vocab = set()
  raise NotImplementedError("STEP 4: Get the vocabulary found in the text")
  return vocab

class UnknownLM(NGramLM):
  def __init__(self,texts,n,vocabulary):
    raise NotImplementedError("STEP 4: Initialize the LM using add-alpha smoothing")
 
  def generate_text(self, length,prompt=[]):
    raise NotImplementedError("STEP 4: Given a prompt (a list of strings) generate length number of strings -- return a list containing all of the text (prompt + generated) ")

  def score_text(self,text):
    raise NotImplementedError("STEP 4: Given a text, return the perplexity of that text ")


truncated = truncate_texts(texts,2)
vocab = get_vocabulary(truncated)
for n in [1,2,3]:
  print(f'Order {n} LM:')
  lm = UnknownLM(texts,n,vocabulary)

  print(' '.join(lm.generate_text(100)))
  print(' '.join(lm.generate_text(100,['palm', 'to'])))

  print(lm.score_text(['IBM','announced','today','that','they','will','be','buying','Google']))
  print(lm.score_text(['palm', 'to', 'palm', 'is', 'holy', 'palmers']))
  print(lm.score_text(['what','do','you','say','to','that','Hamlet']))

#STEP 5

Build more graceful higher order LMs.

* Implement the `InterpolationLM` -- this is like the previous LM, except when scoring and generating it takes in an optional list of interpolants $\lambda_i$
  * NOTE: All interpolants should be 0 < $\lambda_i$ < 1 and should sum to 1
  * The interpolants should be used such that all LMs or order 1 to $n$ are used to provide probabilities, such that $Pr_{interp}(w_i | w_{i-1}...w_0) = \lambda_1 Pr(w) + \lambda_2 Pr(w|w_{i-1}) + ... $
* Implement the `BackOffLM` -- this is like the previous LMs, except the various orders are used in decreasing order -- i.e. given an order 3 `BackOffLM` the trigram model is tried first, then the bigram if the context is missing from the trigram, etc.



In [ ]:

class InterpolationLM(NGramLM):
  def __init__(self,texts,n,vocabulary):
    raise NotImplementedError("STEP 5: Initialize the Interpolation LM")
 
  def generate_text(self, length,interpolants,prompt=[]):
    raise NotImplementedError("STEP 5: Given a prompt (a list of strings) generate length number of strings -- return a list containing all of the text (prompt + generated) ")

  def score_text(self,text,interpolants):
    raise NotImplementedError("STEP 5: Given a text, return the perplexity of that text ")

for interpolants in [[1/3, 1/3, 1/3],[0.1,0.2,0.7],[0.7,0.2,0.1]]:
  print(f'Interpolants = {interpolants}')
  lm = InterpolationLM(texts,3,vocabulary)

  print(' '.join(lm.generate_text(20,interpolants)))
  print(' '.join(lm.generate_text(20,interpolants,['palm', 'to'])))

  print(lm.score_text(['IBM','announced','today','that','they','will','be','buying','Google'],interpolants))
  print(lm.score_text(['palm', 'to', 'palm', 'is', 'holy', 'palmers'],interpolants))
  print(lm.score_text(['what','do','you','say','to','that','Hamlet'],interpolants))

In [ ]:

class BackOffLM(NGramLM):
  def __init__(self,texts,n,vocabulary):
    raise NotImplementedError("STEP 5b: Initialize the Backoff LM")
 
  def generate_text(self, length,prompt=[]):
    raise NotImplementedError("STEP 5b: Given a prompt (a list of strings) generate length number of strings -- return a list containing all of the text (prompt + generated) ")

  def score_text(self,text):
    raise NotImplementedError("STEP 5b: Given a text, return the perplexity of that text ")

lm = BackOffLM(texts,3,vocabulary)

print(' '.join(lm.generate_text(20)))
print(' '.join(lm.generate_text(20,['palm', 'to'])))

print(lm.score_text(['IBM','announced','today','that','they','will','be','buying','Google']))
print(lm.score_text(['palm', 'to', 'palm', 'is', 'holy', 'palmers']))
print(lm.score_text(['what','do','you','say','to','that','Hamlet']))

### Closing Questions:  Compare the different LMs -- 
* #### Which models have the lowest perplexity? 
* #### Which models make a tradeoff for low perplexity in some cases and high in others?  
* #### Which model would you choose  to use?